<a href="https://colab.research.google.com/github/Thiago-Reis-Porto/Python-repository/blob/main/Pycaret_CD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTANTE!! \\
Necessario reiniciar o ambiente apos o pip install

In [ ]:
# %%capture
# !pip install -U pycaret[full] --user
!pip install -U pycaret --user
# #!pip install -U Jinja2 --user

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pycaret-2.3.10-py3-none-any.whl (320 kB)
  Using cached pandas_profiling-3.5.0-py2.py3-none-any.whl (325 kB)
  Using cached scikit_plot-0.3.7-py3-none-any.whl (33 kB)
  Using cached lightgbm-3.3.3-py3-none-manylinux1_x86_64.whl (2.0 MB)
  Using cached mlxtend-0.21.0-py2.py3-none-any.whl (1.3 MB)
  Using cached pyod-1.0.6.tar.gz (141 kB)
  Using cached mlflow-1.30.0-py3-none-any.whl (17.0 MB)
  Using cached scikit_learn-0.23.2-cp37-cp37m-manylinux1_x86_64.whl (6.8 MB)
  Using cached numba-0.54.1-cp37-cp37m-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.3 MB)
  Using cached Boruta-0.3-py3-none-any.whl (56 kB)
  Using cached spacy-2.3.8-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.8 MB)
  Using cached kmodes-0.12.2-py2.py3-none-any.whl (20 kB)
  Using cached umap-learn-0.5.3.tar.gz (88 kB)
  Using cached scipy-1.5.4-cp37-cp37m-manylinux1_x86_64.whl (25.9 MB)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, datetime
import plotly.graph_objects as go
import ephem

In [ ]:
import plotly.express as px

In [ ]:
import logging, sys
logging.disable(sys.maxsize)

#Dados

In [ ]:
!gdown 17QlDvdC_3h73oxSGPDyJe1S6tUnMvy0J

Downloading...
From: https://drive.google.com/uc?id=17QlDvdC_3h73oxSGPDyJe1S6tUnMvy0J
To: /content/df_agrupado_max_prec.csv
100% 80.6k/80.6k [00:00<00:00, 31.8MB/s]


In [ ]:
df_source = pd.read_csv('df_agrupado_max_prec.csv', parse_dates=['datetime'])
df_source = df_source.sort_values('datetime')
df_source.head()

,datetime,max_prec_mm,pilots_mare_meteo_max,pilots_mare_total_max,pilots_mare_res_max,petrobras_nivel_max,comdec_alagamento
0,2016-01-01,9.70,0.25,1.40,0.31,2.979025,0
31,2016-01-02,34.50,0.03,1.08,0.16,2.979025,0
60,2016-01-03,4.90,0.21,1.17,0.20,2.979025,0
91,2016-01-04,0.90,0.01,1.13,-0.02,2.320000,0
121,2016-01-05,0.11,0.49,1.87,0.51,2.300000,0


In [ ]:
df_source.datetime

0      2016-01-01
31     2016-01-02
60     2016-01-03
91     2016-01-04
121    2016-01-05
          ...    
1823   2020-12-28
1824   2020-12-29
1825   2020-12-30
1826   2020-12-31
1827   2021-01-01
Name: datetime, Length: 1828, dtype: datetime64[ns]

##DF1

In [ ]:
# create moving-averages
df_plt = df_source.copy()
df_plt['MA_prec_5'] = df_source['max_prec_mm'].rolling(5).mean()
df_plt['MA_prec_3'] = df_source['max_prec_mm'].rolling(3).mean()

# plot 
fig = px.line(df_plt, x='datetime', y=['max_prec_mm', 'MA_prec_5', 'MA_prec_3'], title='max_prec_mm', template = 'plotly_dark')
fig.show()

In [ ]:
Y = 2000 # dummy leap year to allow input X-02-29 (leap day)
seasons = [('Verão', (date(Y,  1,  1),  date(Y,  3, 20))),
           ('Outono', (date(Y,  3, 21),  date(Y,  6, 20))),
           ('Inverno', (date(Y,  6, 21),  date(Y,  9, 22))),
           ('Primavera', (date(Y,  9, 23),  date(Y, 12, 21))),
           ('Verão', (date(Y, 12, 22),  date(Y, 12, 31)))]

def get_season(now):
    now = now.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= now.date() <= end)

In [ ]:
# set timestamp to index
df = df_source.copy()
df['estacao'] = df['datetime'].apply(lambda x: get_season(x))
df.set_index('datetime', drop=True, inplace=True)

# creature features from date
# df.assign(
#   day =  [i.day for i in df.index],
#   day_of_year = [i.dayofyear for i in df.index],
#   week_of_year = [i.weekofyear for i in df.index])

df.head()

,max_prec_mm,pilots_mare_meteo_max,pilots_mare_total_max,pilots_mare_res_max,petrobras_nivel_max,comdec_alagamento,estacao
datetime,,,,,,,
2016-01-01,9.70,0.25,1.40,0.31,2.979025,0,Verão
2016-01-02,34.50,0.03,1.08,0.16,2.979025,0,Verão
2016-01-03,4.90,0.21,1.17,0.20,2.979025,0,Verão
2016-01-04,0.90,0.01,1.13,-0.02,2.320000,0,Verão
2016-01-05,0.11,0.49,1.87,0.51,2.300000,0,Verão


#DF2

In [ ]:
df2 = df.copy()
df2['fase_lua'] = df2.index.map(lambda x: ephem.Moon(x).moon_phase)
df2

,max_prec_mm,pilots_mare_meteo_max,pilots_mare_total_max,pilots_mare_res_max,petrobras_nivel_max,comdec_alagamento,estacao,fase_lua
datetime,,,,,,,,
2016-01-01,9.700,0.250000,1.40,0.31,2.979025,0,Verão,0.616039
2016-01-02,34.500,0.030000,1.08,0.16,2.979025,0,Verão,0.523040
2016-01-03,4.900,0.210000,1.17,0.20,2.979025,0,Verão,0.429505
2016-01-04,0.900,0.010000,1.13,-0.02,2.320000,0,Verão,0.337928
2016-01-05,0.110,0.490000,1.87,0.51,2.300000,0,Verão,0.250988
...,...,...,...,...,...,...,...,...
2020-12-28,14.800,-0.100000,1.40,0.03,2.460000,0,Verão,0.955922
2020-12-29,9.000,-0.070000,1.45,0.10,2.410000,0,Verão,0.987177
2020-12-30,1.400,-0.060000,1.47,0.10,2.480000,0,Verão,0.999653


#DF3

In [ ]:
!gdown 1706QYNpmOM2kcb13wEx9KeHtvkT7ndlA

Downloading...
From: https://drive.google.com/uc?id=1706QYNpmOM2kcb13wEx9KeHtvkT7ndlA
To: /content/ECOVIAS_precip_diaria.csv
100% 25.7k/25.7k [00:00<00:00, 25.7MB/s]


In [ ]:
!gdown 17OSATZ1TBnijAauehOXkCSo9sAAp6CNQ

Downloading...
From: https://drive.google.com/uc?id=17OSATZ1TBnijAauehOXkCSo9sAAp6CNQ
To: /content/GFS_CubataoEcovias13_apcp_acum.csv
100% 155k/155k [00:00<00:00, 61.4MB/s]


In [ ]:
eco = pd.read_csv('ECOVIAS_precip_diaria.csv', parse_dates=True, index_col='datetime')
eco

,ecovias_prec_mm
datetime,
2018-06-13,1.2
2018-06-14,2.4
2018-06-15,8.3
2018-06-16,5.9
2018-06-17,0.3
...,...
2022-07-02,0.0
2022-07-03,0.0
2022-07-04,0.0


In [ ]:
gfs_eco = pd.read_csv('GFS_CubataoEcovias13_apcp_acum.csv', 
                           parse_dates=True, 
                           index_col='time')

In [ ]:
t = pd.to_datetime(gfs_eco.index, utc=True)
gfs_eco.index = pd.to_datetime(t.strftime('%d/%m/%Y'))
gfs_eco

,estacao,latitude,longitude,precipitacao_diaria,classes_precipitacao
time,,,,,
2015-12-31,CubataoEcovias13,-24.0,313.5,2.30,Cf
2016-01-01,CubataoEcovias13,-24.0,313.5,10.50,CM
2016-02-01,CubataoEcovias13,-24.0,313.5,33.30,CF
2016-03-01,CubataoEcovias13,-24.0,313.5,3.70,Cf
2016-04-01,CubataoEcovias13,-24.0,313.5,0.31,DS
...,...,...,...,...,...
2022-05-29,CubataoEcovias13,-24.0,313.5,0.00,DS
2022-05-30,CubataoEcovias13,-24.0,313.5,1.69,DS
2022-05-31,CubataoEcovias13,-24.0,313.5,22.12,CM


In [ ]:
df3 = df2.copy()
df3['prec_mm_ecv'] = eco
df3['prec_mm_ecv'] = df3['prec_mm_ecv'].fillna(gfs_eco.precipitacao_diaria)
df3.head()

,max_prec_mm,pilots_mare_meteo_max,pilots_mare_total_max,pilots_mare_res_max,petrobras_nivel_max,comdec_alagamento,estacao,fase_lua,prec_mm_ecv
datetime,,,,,,,,,
2016-01-01,9.70,0.25,1.40,0.31,2.979025,0,Verão,0.616039,10.50
2016-01-02,34.50,0.03,1.08,0.16,2.979025,0,Verão,0.523040,0.00
2016-01-03,4.90,0.21,1.17,0.20,2.979025,0,Verão,0.429505,4.60
2016-01-04,0.90,0.01,1.13,-0.02,2.320000,0,Verão,0.337928,1.92
2016-01-05,0.11,0.49,1.87,0.51,2.300000,0,Verão,0.250988,0.60


In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1828 entries, 2016-01-01 to 2021-01-01
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   max_prec_mm            1828 non-null   float64
 1   pilots_mare_meteo_max  1828 non-null   float64
 2   pilots_mare_total_max  1828 non-null   float64
 3   pilots_mare_res_max    1828 non-null   float64
 4   petrobras_nivel_max    1828 non-null   float64
 5   comdec_alagamento      1828 non-null   int64  
 6   estacao                1828 non-null   object 
 7   fase_lua               1828 non-null   float64
 8   prec_mm_ecv            1828 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 142.8+ KB


#DF4 apenas dados de modelo na prec

In [ ]:
!gdown 1gboZDexg0edXQfm2nnpKFIRfstBYuro4
!gdown 1h3jBavGmwEg6vMCSFZoLesU4ns4qXBw7
!gdown 1hJFJWrYF8T7sb1tP3RN-lj9oeIz2QyfH


Downloading...
From: https://drive.google.com/uc?id=1gboZDexg0edXQfm2nnpKFIRfstBYuro4
To: /content/GFS_CubataoCota200_apcp_acum.csv
100% 115k/115k [00:00<00:00, 58.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1h3jBavGmwEg6vMCSFZoLesU4ns4qXBw7
To: /content/GFS_CubataoJardimCasqueiro_apcp_acum.csv
100% 134k/134k [00:00<00:00, 36.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1hJFJWrYF8T7sb1tP3RN-lj9oeIz2QyfH
To: /content/GFS_CubataoParqueFernandoJorge_apcp_acum.csv
100% 143k/143k [00:00<00:00, 62.5MB/s]


In [ ]:
gfs_cota_200 = pd.read_csv('GFS_CubataoCota200_apcp_acum.csv', 
                           parse_dates=True, 
                           index_col='time')

gfs_casqueiro = pd.read_csv('GFS_CubataoJardimCasqueiro_apcp_acum.csv', 
                           parse_dates=True, 
                           index_col='time')

gfs_fernandojorge = pd.read_csv('GFS_CubataoParqueFernandoJorge_apcp_acum.csv', 
                           parse_dates=True, 
                           index_col='time')


In [ ]:
gfs_eco = pd.read_csv('GFS_CubataoEcovias13_apcp_acum.csv', 
                           parse_dates=True, 
                           index_col='time')
gfs_eco

,estacao,latitude,longitude,precipitacao_diaria,classes_precipitacao
time,,,,,
2015-12-31 00:00:00-02:00,CubataoEcovias13,-24.0,313.5,2.30,Cf
2016-01-01 00:00:00-02:00,CubataoEcovias13,-24.0,313.5,10.50,CM
2016-01-02 00:00:00-02:00,CubataoEcovias13,-24.0,313.5,33.30,CF
2016-01-03 00:00:00-02:00,CubataoEcovias13,-24.0,313.5,3.70,Cf
2016-01-04 00:00:00-02:00,CubataoEcovias13,-24.0,313.5,0.31,DS
...,...,...,...,...,...
2022-05-29 00:00:00-03:00,CubataoEcovias13,-24.0,313.5,0.00,DS
2022-05-30 00:00:00-03:00,CubataoEcovias13,-24.0,313.5,1.69,DS
2022-05-31 00:00:00-03:00,CubataoEcovias13,-24.0,313.5,22.12,CM


In [ ]:
gfs_fernandojorge.precipitacao_diaria

time
2015-12-31     0.20
2016-01-01     9.70
2016-01-02    34.50
2016-01-03     4.90
2016-01-04     0.90
              ...  
2022-05-29     0.00
2022-05-30     0.38
2022-05-31    21.56
2022-06-01    11.50
2022-06-02     5.69
Name: precipitacao_diaria, Length: 2346, dtype: float64

In [ ]:
df_modelo = df.drop('max_prec_mm', 1)
df_modelo['gfs_prec_mm'] = gfs_fernandojorge.precipitacao_diaria
df_modelo 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



,pilots_mare_meteo_max,pilots_mare_total_max,pilots_mare_res_max,petrobras_nivel_max,comdec_alagamento,estacao,gfs_prec_mm
datetime,,,,,,,
2016-01-01,0.250000,1.40,0.31,2.979025,0,Verão,9.70
2016-01-02,0.030000,1.08,0.16,2.979025,0,Verão,34.50
2016-01-03,0.210000,1.17,0.20,2.979025,0,Verão,4.90
2016-01-04,0.010000,1.13,-0.02,2.320000,0,Verão,0.90
2016-01-05,0.490000,1.87,0.51,2.300000,0,Verão,0.11
...,...,...,...,...,...,...,...
2020-12-28,-0.100000,1.40,0.03,2.460000,0,Verão,14.31
2020-12-29,-0.070000,1.45,0.10,2.410000,0,Verão,5.56
2020-12-30,-0.060000,1.47,0.10,2.480000,0,Verão,3.75


#Pycaret detecção de anomalia


In [ ]:
from pycaret.anomaly import *

In [ ]:
s = setup(df, 
          session_id = 123,
          silent=True)

,Description,Value
0,session_id,123
1,Original Data,"(1828, 7)"
2,Missing Values,False
3,Numeric Features,5
4,Categorical Features,2
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(1828, 11)"
9,CPU Jobs,-1


##isolation forest

In [ ]:
# train model
iforest = create_model('iforest', fraction = 0.1)
iforest_results = assign_model(iforest)
iforest_results.head()

,max_prec_mm,pilots_mare_meteo_max,pilots_mare_total_max,pilots_mare_res_max,petrobras_nivel_max,comdec_alagamento,estacao,Anomaly,Anomaly_Score
datetime,,,,,,,,,
2016-01-01,9.70,0.25,1.40,0.31,2.979025,0,Verão,0,-0.079769
2016-01-02,34.50,0.03,1.08,0.16,2.979025,0,Verão,0,-0.039013
2016-01-03,4.90,0.21,1.17,0.20,2.979025,0,Verão,0,-0.079352
2016-01-04,0.90,0.01,1.13,-0.02,2.320000,0,Verão,0,-0.073934
2016-01-05,0.11,0.49,1.87,0.51,2.300000,0,Verão,0,-0.001241


In [ ]:
# check anomalies
iforest_results[iforest_results['Anomaly'] == 1].head()

,max_prec_mm,pilots_mare_meteo_max,pilots_mare_total_max,pilots_mare_res_max,petrobras_nivel_max,comdec_alagamento,estacao,Anomaly,Anomaly_Score
datetime,,,,,,,,,
2016-01-08,0.0,0.091029,0.95,-0.31,2.230000,0,Verão,1,0.014640
2016-01-11,39.0,-0.350000,1.27,-0.10,2.340000,0,Verão,1,0.048074
2016-01-14,10.6,0.050000,1.44,0.16,2.979025,1,Verão,1,0.048291
2016-02-01,0.0,0.220000,1.17,0.26,2.979025,1,Verão,1,0.058629
2016-02-14,0.0,-0.240000,0.87,-0.21,2.979025,0,Verão,1,0.007814


In [ ]:
iforest_results[iforest_results['Anomaly'] == 1].shape

(183, 9)

In [ ]:
# plot value on y-axis and date on x-axis
fig = px.line(iforest_results, x=iforest_results.index, y='max_prec_mm', title='max_prec_mm - UNSUPERVISED ANOMALY DETECTION', template = 'plotly_dark')
# create list of outlier_dates
outlier_dates = iforest_results[iforest_results['Anomaly'] == 1].index
# obtain y value of anomalies to plot
y_values = [iforest_results.loc[i]['max_prec_mm'] for i in outlier_dates]
fig.add_trace(go.Scatter(x=outlier_dates, y=y_values, mode = 'markers', 
                name = 'Anomaly', 
                marker=dict(color='red',size=10)))
        
fig.show()

##knn


In [ ]:
# train model
knn = create_model('knn', fraction = 0.1)
knn_results = assign_model(knn)
knn_results.head()

,max_prec_mm,pilots_mare_meteo_max,pilots_mare_total_max,pilots_mare_res_max,petrobras_nivel_max,comdec_alagamento,estacao,Anomaly,Anomaly_Score
datetime,,,,,,,,,
2016-01-01,9.70,0.25,1.40,0.31,2.979025,0,Verão,0,0.564361
2016-01-02,34.50,0.03,1.08,0.16,2.979025,0,Verão,1,1.538614
2016-01-03,4.90,0.21,1.17,0.20,2.979025,0,Verão,0,0.668996
2016-01-04,0.90,0.01,1.13,-0.02,2.320000,0,Verão,0,0.430775
2016-01-05,0.11,0.49,1.87,0.51,2.300000,0,Verão,0,0.487852


In [ ]:
# check anomalies
knn_results[knn_results['Anomaly'] == 1].head()

,max_prec_mm,pilots_mare_meteo_max,pilots_mare_total_max,pilots_mare_res_max,petrobras_nivel_max,comdec_alagamento,estacao,Anomaly,Anomaly_Score
datetime,,,,,,,,,
2016-01-02,34.50,0.03,1.08,0.16,2.979025,0,Verão,1,1.538614
2016-01-11,39.00,-0.35,1.27,-0.10,2.340000,0,Verão,1,1.971563
2016-01-12,24.50,0.05,1.67,0.18,2.450000,0,Verão,1,1.496830
2016-01-14,10.60,0.05,1.44,0.16,2.979025,1,Verão,1,1.519507
2016-01-15,41.12,0.14,1.41,0.21,2.979025,0,Verão,1,1.723731


In [ ]:
# plot value on y-axis and date on x-axis
fig = px.line(knn_results, x=knn_results.index, y='max_prec_mm', title='max_prec_mm - UNSUPERVISED ANOMALY DETECTION', template = 'plotly_dark')
# create list of outlier_dates
outlier_dates = knn_results[knn_results['Anomaly'] == 1].index
# obtain y value of anomalies to plot
y_values = [knn_results.loc[i]['max_prec_mm'] for i in outlier_dates]
fig.add_trace(go.Scatter(x=outlier_dates, y=y_values, mode = 'markers', 
                name = 'Anomaly', 
                marker=dict(color='red',size=10)))
        
fig.show()

#Pycaret Classificação

In [ ]:
df.tail(1)

,max_prec_mm,pilots_mare_meteo_max,pilots_mare_total_max,pilots_mare_res_max,petrobras_nivel_max,comdec_alagamento,estacao
datetime,,,,,,,
2021-01-01,67.4,0.091029,1.7,0.43,4.07,0,Verão


In [ ]:
df.query('comdec_alagamento == 1')

,max_prec_mm,pilots_mare_meteo_max,pilots_mare_total_max,pilots_mare_res_max,petrobras_nivel_max,comdec_alagamento,estacao
datetime,,,,,,,
2016-01-14,10.600,0.050000,1.440000,0.16000,2.979025,1,Verão
2016-02-01,0.000,0.220000,1.170000,0.26000,2.979025,1,Verão
2016-04-28,0.320,0.091029,1.780000,0.52000,2.310000,1,Outono
2016-08-22,5.800,0.091029,1.480531,0.18305,2.490000,1,Inverno
2016-12-21,45.861,0.090000,1.220000,0.27000,3.480000,1,Primavera
2017-03-13,14.196,0.190000,1.840000,0.32000,3.500000,1,Verão
2018-04-15,158.068,0.091029,1.980000,0.34000,4.960000,1,Outono
2018-11-08,18.310,0.091029,2.400000,0.84000,2.310000,1,Primavera
2019-04-02,214.363,0.091029,2.100000,0.60000,5.150000,1,Outono


In [ ]:
from pycaret.classification import *

In [ ]:
# df.query()

In [ ]:
df.loc[:'2018'].shape[0] / 10

109.6

In [ ]:
trn = df.loc[:'2018']
tst = df.loc['2019':]

In [ ]:
tst.query('comdec_alagamento == 1')

,max_prec_mm,pilots_mare_meteo_max,pilots_mare_total_max,pilots_mare_res_max,petrobras_nivel_max,comdec_alagamento,estacao
datetime,,,,,,,
2019-04-02,214.363,0.091029,2.10,0.6,5.15,1,Outono
2020-03-01,16.955,0.340000,1.46,0.5,3.28,1,Verão


In [ ]:
tst.loc['2020-02-25':'2020-03-05']

,max_prec_mm,pilots_mare_meteo_max,pilots_mare_total_max,pilots_mare_res_max,petrobras_nivel_max,comdec_alagamento,estacao
datetime,,,,,,,
2020-02-25,16.400,0.00,1.63,0.06,3.61,0,Verão
2020-02-26,67.416,0.33,1.59,0.42,3.80,0,Verão
2020-02-27,55.842,0.46,1.97,0.59,4.03,0,Verão
2020-02-28,26.200,0.36,1.75,0.40,3.59,0,Verão
2020-02-29,58.400,0.17,1.46,0.19,4.42,0,Verão
2020-03-01,16.955,0.34,1.46,0.50,3.28,1,Verão
2020-03-02,81.962,0.05,1.29,0.15,3.38,0,Verão
2020-03-03,88.213,0.10,1.21,0.18,5.06,0,Verão
2020-03-04,0.400,0.64,1.95,0.43,3.69,0,Verão


In [ ]:
tst.loc['2019-03']

,max_prec_mm,pilots_mare_meteo_max,pilots_mare_total_max,pilots_mare_res_max,petrobras_nivel_max,comdec_alagamento,estacao
datetime,,,,,,,
2019-03-01,17.120,-0.270000,1.18,-0.11,2.47,0,Verão
2019-03-02,61.372,0.091029,1.58,0.52,3.82,0,Verão
2019-03-03,26.209,0.091029,1.31,0.15,3.72,0,Verão
2019-03-04,0.600,0.070000,1.72,0.20,3.06,0,Verão
2019-03-05,1.500,0.280000,1.86,0.36,2.50,0,Verão
2019-03-06,34.733,0.180000,1.76,0.17,3.55,0,Verão
2019-03-07,0.250,0.370000,2.09,0.56,2.60,0,Verão
2019-03-08,10.171,0.390000,1.95,0.58,3.25,0,Verão
2019-03-09,5.800,0.230000,1.62,0.37,2.38,0,Verão


In [ ]:
s = setup(data = trn,
          test_data = tst,
          target = 'comdec_alagamento',
          fold_strategy = 'timeseries',
          data_split_shuffle = False,
          #numeric_features = ['Year', 'Series'],
          fold = 10,
          #transform_target = True,
          #normalize=True,
          silent=True,
          session_id = 123)

,Description,Value
0,session_id,123
1,Target,comdec_alagamento
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(1096, 7)"
5,Missing Values,False
6,Numeric Features,5
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9939,0.3622,0.0,0.000,0.0000,NaN,0.0000,0.192
knn,K Neighbors Classifier,0.9939,0.2867,0.0,0.000,0.0000,NaN,0.0000,0.114
ridge,Ridge Classifier,0.9939,0.0000,0.0,0.000,0.0000,NaN,0.0000,0.013
rf,Random Forest Classifier,0.9939,0.3046,0.0,0.000,0.0000,NaN,0.0000,0.464
ada,Ada Boost Classifier,0.9939,0.3454,0.0,0.000,0.0000,NaN,0.0000,0.098
et,Extra Trees Classifier,0.9939,0.2347,0.0,0.000,0.0000,NaN,0.0000,0.443
dummy,Dummy Classifier,0.9939,0.3000,0.0,0.000,0.0000,NaN,0.0000,0.010
lightgbm,Light Gradient Boosting Machine,0.9929,0.3398,0.0,0.000,0.0000,NaN,-0.0010,0.121
gbc,Gradient Boosting Classifier,0.9889,0.3342,0.0,0.000,0.0000,NaN,-0.0020,0.107
lda,Linear Discriminant Analysis,0.9889,0.3857,0.0,0.000,0.0000,NaN,-0.0035,0.015


In [ ]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
class_weight = {0: 1,
                1: 40.}
lr = create_model('lr', class_weight=class_weight)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9798,0.4898,0.0,0.0000,0.0000,-0.0102,-0.0102
1,0.9091,0.4286,0.0,0.0000,0.0000,-0.0183,-0.0300
2,0.8990,0.2245,0.0,0.0000,0.0000,-0.0185,-0.0319
3,0.8788,0.7245,0.0,0.0000,0.0000,-0.0189,-0.0357
4,0.9192,0.0000,0.0,0.0000,0.0000,0.0000,0.0000
5,0.9394,0.0000,0.0,0.0000,0.0000,0.0000,0.0000
6,0.6364,0.0000,0.0,0.0000,0.0000,0.0000,0.0000
7,0.9293,0.9796,1.0,0.1250,0.2222,0.2080,0.3407
8,0.9495,0.0000,0.0,0.0000,0.0000,0.0000,0.0000


In [ ]:
evaluate_model(lr)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
p = predict_model(lr, tst)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.9098,0.9719,1.0,0.0294,0.0571,0.0521,0.1636


In [ ]:
p.loc['2020-03-03']

max_prec_mm              88.213
pilots_mare_meteo_max       0.1
pilots_mare_total_max      1.21
pilots_mare_res_max        0.18
petrobras_nivel_max        5.06
comdec_alagamento             0
estacao                   Verão
Label                         1
Score                    0.8048
Name: 2020-03-03 00:00:00, dtype: object

In [ ]:
class_weight = {0: 1.,
                1: 37.}
ridge = create_model('ridge', class_weight=class_weight)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9495,0.0,0.0,0.00,0.0000,-0.0164,-0.0207
1,0.8990,0.0,0.0,0.00,0.0000,-0.0185,-0.0319
2,0.8990,0.0,0.0,0.00,0.0000,-0.0185,-0.0319
3,0.8990,0.0,0.0,0.00,0.0000,-0.0185,-0.0319
4,0.9091,0.0,0.0,0.00,0.0000,0.0000,0.0000
5,0.9596,0.0,0.0,0.00,0.0000,0.0000,0.0000
6,0.6970,0.0,0.0,0.00,0.0000,0.0000,0.0000
7,0.9596,0.0,1.0,0.20,0.3333,0.3219,0.4380
8,0.9798,0.0,0.0,0.00,0.0000,0.0000,0.0000


In [ ]:
evaluate_model(ridge)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
class_weight = {0: 0.1,
                1: 99.}
dt = create_model('dt', class_weight=class_weight)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9697,0.4898,0.0,0.0,0.0,-0.0137,-0.0145
1,0.9899,0.5000,0.0,0.0,0.0,0.0000,0.0000
2,0.9798,0.4949,0.0,0.0,0.0,-0.0102,-0.0102
3,0.9899,0.5000,0.0,0.0,0.0,0.0000,0.0000
4,0.9899,0.0000,0.0,0.0,0.0,0.0000,0.0000
5,0.9798,0.0000,0.0,0.0,0.0,0.0000,0.0000
6,1.0000,0.0000,0.0,0.0,0.0,NaN,0.0000
7,0.9899,0.5000,0.0,0.0,0.0,0.0000,0.0000
8,0.9798,0.0000,0.0,0.0,0.0,0.0000,0.0000


In [ ]:
evaluate_model(dt)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

##DF gfs

In [ ]:
trn = df_modelo.loc[:'2018']
tst = df_modelo.loc['2019':]

In [ ]:
s = setup(data = trn,
          test_data = tst,
          target = 'comdec_alagamento',
          fold_strategy = 'timeseries',
          data_split_shuffle = False,
          #numeric_features = ['Year', 'Series'],
          fold = 10,
          #transform_target = True,
          #normalize=True,
          silent=True,
          session_id = 123)

In [ ]:
class_weight = {0: 1,
                1: 20.}
lr = create_model('lr', class_weight=class_weight)

In [ ]:
evaluate_model(lr)

##DF2

In [ ]:
trn = df2.loc[:'2018']
tst = df2.loc['2019':]

In [ ]:
s = setup(data = trn,
          test_data = tst,
          target = 'comdec_alagamento',
          fold_strategy = 'timeseries',
          data_split_shuffle = False,
          fold = 10,
          silent=True,
          session_id = 123)

In [ ]:
class_weight = {0: 1.,
                1: 40.}
lr = create_model('lr', class_weight=class_weight)

In [ ]:
evaluate_model(lr)

In [ ]:
class_weight = {0: 1.,
                1: 40.}
ridge = create_model('ridge', class_weight=class_weight)

In [ ]:
evaluate_model(ridge)

In [ ]:
s = setup(data = trn[['pilots_mare_meteo_max', 'pilots_mare_res_max', 'comdec_alagamento']],
          test_data = tst[['pilots_mare_meteo_max', 'pilots_mare_res_max', 'comdec_alagamento']],
          target = 'comdec_alagamento',
          fold_strategy = 'timeseries',
          data_split_shuffle = False,
          #numeric_features = ['Year', 'Series'],
          fold = 10,
          #transform_target = True,
          #normalize=True,
          silent=True,
          session_id = 123)

In [ ]:
class_weight = {0: 1.,
                1: 65.}
lr = create_model('lr', class_weight=class_weight)

In [ ]:
evaluate_model(lr)

In [ ]:
class_weight = {0: .5,
                1: 90.}
dt = create_model('dt', class_weight=class_weight)

In [ ]:
evaluate_model(dt)

##DF3


In [ ]:
trn = df3.loc[:'2018']
tst = df3.loc['2019':]

In [ ]:
s = setup(data = trn,
          test_data = tst,
          target = 'comdec_alagamento',
          fold_strategy = 'timeseries',
          data_split_shuffle = False,
          fold = 10,
          silent=True,
          session_id = 123)

In [ ]:
class_weight = {0: 1.,
                1: 4.}
lr = create_model('lr', class_weight=class_weight)

In [ ]:
evaluate_model(lr)

In [ ]:
class_weight = {0: 1.,
                1: 40.}
ridge = create_model('ridge', class_weight=class_weight)

In [ ]:
evaluate_model(ridge)

In [ ]:
s = setup(data = trn[['pilots_mare_meteo_max', 'pilots_mare_res_max', 'comdec_alagamento']],
          test_data = tst[['pilots_mare_meteo_max', 'pilots_mare_res_max', 'comdec_alagamento']],
          target = 'comdec_alagamento',
          fold_strategy = 'timeseries',
          data_split_shuffle = False,
          #numeric_features = ['Year', 'Series'],
          fold = 10,
          #transform_target = True,
          #normalize=True,
          silent=True,
          session_id = 123)

In [ ]:
class_weight = {0: 1.,
                1: 65.}
lr = create_model('lr', class_weight=class_weight)

In [ ]:
evaluate_model(lr)

In [ ]:
class_weight = {0: .5,
                1: 90.}
dt = create_model('dt', class_weight=class_weight)

In [ ]:
evaluate_model(dt)